<a href="https://colab.research.google.com/github/WilliamRossCrane/Code-Challange-Solutions/blob/main/books_scraping_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 Books to Scrape — Web Scraping, Cleaning, and SQL ETL Example

This notebook demonstrates a simple end-to-end data pipeline:
- **Scrape** product data (title, price, availability) from the [Books to Scrape](http://books.toscrape.com/) practice site.
- **Clean & transform** the raw text data using Python and pandas.
- **Store** the processed data in an SQLite database.
- **Query** the data using SQL for basic analysis.

This is a beginner-friendly, industry-relevant example of an ETL (Extract, Transform, Load) workflow, suitable for practicing web scraping, data wrangling, and SQL integration.

In [1]:
# Install dependencies if needed
!pip install beautifulsoup4

import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

In [7]:
# ✅ Site URL
url = 'http://books.toscrape.com/catalogue/page-1.html'

# ✅ Fetch page and force UTF-8 decoding
response = requests.get(url)
response.encoding = 'utf-8'  # This fixes weird £ symbols!

print("Status:", response.status_code)

# ✅ Parse HTML
soup = BeautifulSoup(response.text, 'html.parser')

Status: 200


In [8]:
# ✅ Find all product pods
books = soup.find_all('article', class_='product_pod')

# ✅ Extract data cleanly
titles = []
prices = []
availability = []

for book in books:
    title = book.h3.a['title']
    # Strip spaces + fix odd chars if any:
    price = book.find('p', class_='price_color').text.strip()
    stock = book.find('p', class_='instock availability').text.strip()

    titles.append(title)
    prices.append(price)
    availability.append(stock)

# ✅ Preview to confirm correct £ signs
print("Sample titles:", titles[:5])
print("Sample prices:", prices[:5])
print("Sample availability:", availability[:5])


Sample titles: ['A Light in the Attic', 'Tipping the Velvet', 'Soumission', 'Sharp Objects', 'Sapiens: A Brief History of Humankind']
Sample prices: ['£51.77', '£53.74', '£50.10', '£47.82', '£54.23']
Sample availability: ['In stock', 'In stock', 'In stock', 'In stock', 'In stock']


In [9]:
# Clean £ and convert to float
prices = [float(p.replace('£', '').replace('Â', '').strip()) for p in prices]

In [10]:
# ✅ Create a pandas DataFrame to store the scraped book data in tabular format
df = pd.DataFrame({
    'Title': titles,
    'Price': prices,
    'Availability': availability
})

# ✅ Display the first few rows to verify the data
df.head()

,Title,Price,Availability
0,A Light in the Attic,51.77,In stock
1,Tipping the Velvet,53.74,In stock
2,Soumission,50.10,In stock
3,Sharp Objects,47.82,In stock
4,Sapiens: A Brief History of Humankind,54.23,In stock


In [11]:
print(df.describe())

# ✅ How many are in stock?
df['Availability'].value_counts()

           Price
count  20.000000
mean   38.048500
std    15.135231
min    13.990000
25%    22.637500
50%    41.380000
75%    51.865000
max    57.250000


,count
Availability,
In stock,20


In [12]:
# ✅ Create SQLite database in Colab
conn = sqlite3.connect('books.db')

# ✅ Save DataFrame to SQL table named 'books'
df.to_sql('books', conn, if_exists='replace', index=False)

print("Saved to books.db")

Saved to books.db


In [13]:
# ✅ Use SQL to get average price
query = "SELECT AVG(Price) as avg_price FROM books"
avg_price = pd.read_sql_query(query, conn)
print(avg_price)

# G✅ et books over £50
expensive_books = pd.read_sql_query("SELECT * FROM books WHERE Price > 50", conn)
expensive_books

   avg_price
0    38.0485


,Title,Price,Availability
0,A Light in the Attic,51.77,In stock
1,Tipping the Velvet,53.74,In stock
2,Soumission,50.10,In stock
3,Sapiens: A Brief History of Humankind,54.23,In stock
4,The Black Maria,52.15,In stock
5,Scott Pilgrim's Precious Little Life (Scott Pi...,52.29,In stock
6,Our Band Could Be Your Life: Scenes from the A...,57.25,In stock
7,Libertarianism for Beginners,51.33,In stock


In [14]:
conn.close()